In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os

In [2]:
LOCAL_DATA_PATH='data'
TRACES=['youtube', 'bilibili']
APP2APP_TYPE = {
    'youtube':'streaming video',
    'bilibili':'streaming video'
}
FEATURE_LEVEL={'flow':'flow', 'pkt':'pkt'}
FEATURE_FILE_TYPE = 'csv'
PROTOCOLS=['udp', 'tcp']

In [3]:
def get_tcp_udp_info(trace_name, feature_level, file_type):
    udp_filename = os.path.join(LOCAL_DATA_PATH, '{trace_name}_udp_{feature_level}.{file_type}'
                                .format(trace_name=trace_name, feature_level=feature_level, file_type=file_type))
    tcp_filename = os.path.join(LOCAL_DATA_PATH, '{trace_name}_tcp_{feature_level}.{file_type}'
                                .format(trace_name=trace_name, feature_level=feature_level, file_type=file_type))
    udp_info, tcp_info = pd.read_csv(udp_filename), pd.read_csv(tcp_filename)
    return tcp_info.shape[0], udp_info.shape[0]

def get_app_type_feature(trace_name, file_type):
    tcp_flow_num, udp_flow_num = get_tcp_udp_info(trace_name, FEATURE_LEVEL['flow'], FEATURE_FILE_TYPE)
    category = APP2APP_TYPE[trace_name]
    return {
        'category':category,
        'total number flow': tcp_flow_num+udp_flow_num,
        'tcp flow number': tcp_flow_num,
        'udp flow number': udp_flow_num
    }

In [8]:
flow_per_app_type = pd.DataFrame(columns=['category', 'total number flow', 'tcp flow number', 'udp flow number'])
for trace in TRACES:
    flow_feature = get_app_type_feature(trace, FEATURE_FILE_TYPE)
    flow_per_app_type = flow_per_app_type.append(flow_feature, ignore_index=True)
        
flow_per_app_type.groupby('category').sum()912

,category,total number flow,tcp flow number,udp flow number
0,streaming video,912,748,164
1,streaming video,593,579,14
